In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [ ]:
# Load LLaMA 7B model and tokenizer
model_name = "huggingface/llama-7b"
model = LlamaForCausalLM.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

In [ ]:
# Load dataset
dataset = load_dataset("gbharti/finance-alpaca")

In [ ]:
# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=16,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target modules in the transformer layers
    lora_dropout=0.1,
)

In [ ]:
# Apply LoRA configuration to the model
model = get_peft_model(model, lora_config)

In [ ]:
# Tokenization settings
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Mixed precision training
    push_to_hub=False,  # Set to True if you want to push to Hugging Face Hub
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Start fine-tuning
trainer.train()

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./finetuned-llama-7b")
tokenizer.save_pretrained("./finetuned-llama-7b")